# Giriş işlemleri

In [ ]:
%%capture
!pip install transformers torch numpy pandas scikit-learn matplotlib seaborn

In [ ]:
import os

In [ ]:
def is_colab():
    try:
        import google.colab
        return True
    except ImportError:
        return False

In [ ]:
# Kök dizin belirleme
if is_colab():
    from google.colab import drive
    drive.mount('/content/drive')
    kok_dizin = "/content/drive/MyDrive/TurkNLP-ModelBench"
else:
    kok_dizin = os.getcwd()
print(f"Kök dizin: {kok_dizin}\n Not: eğer colab kullanıyorsanız, dizini değiştirmeniz gerekir.")

In [ ]:
import sys
sys.path.append(kok_dizin)

In [ ]:
from dizin_yardimcisi import (gomme_sonucu_ekle, hesaplanan_gomme_sayisi_al, gomme_bilgisi_oku, veri_kumesi_yukle,
                               veri_kumesi_dizin_al, veri_kumesi_adlari_listele, gomme_basarimi_var_mi, gomme_basarimi_kaydet)

# Veri Kümeleri Okuma

## Veri kümeleri dizini alma

In [ ]:
veri_kumeleri_dizini = veri_kumesi_dizin_al()
print(f"Veri kümeleri dizini: {veri_kumeleri_dizini}")

## Veri kümesi adlarını alma

In [ ]:
veri_kumesi_adlari = veri_kumesi_adlari_listele(veri_kumeleri_dizini)
print(f"Veri kümeleri adları: {veri_kumesi_adlari}")

## Veri kümelerini oku

In [ ]:
sinama_kumeleri = {}
egiti_kumeleri = {}
print("Tüm veri kümeleri yükleniyor...")
for veri_kumesi_adi in veri_kumesi_adlari:
    sinama_kumeleri[veri_kumesi_adi] = veri_kumesi_yukle(veri_kumesi_adi, egitim_kumesi_mi=False, klasor_yolu=veri_kumeleri_dizini)
    print(f"{veri_kumesi_adi}: {sinama_kumeleri[veri_kumesi_adi]}")
    egiti_kumeleri[veri_kumesi_adi] = veri_kumesi_yukle(veri_kumesi_adi, egitim_kumesi_mi=True, klasor_yolu=veri_kumeleri_dizini)
    print(f"{veri_kumesi_adi}: {egiti_kumeleri[veri_kumesi_adi]}")
veri_kumeleri = {"sinama_kumeleri": sinama_kumeleri, "egiti_kumeleri": egiti_kumeleri}
print("Tüm veri kümeleri yüklendi.")


# Gömme İşlemleri

## Gömme hesaplama fonksiyonu

In [ ]:
from transformers import AutoModel, AutoTokenizer
import torch
import numpy as np
import pandas as pd

In [ ]:
def get_token_embedding(model: AutoModel, tokenizer: AutoTokenizer, text: str, device_type: str = "cuda") -> np.ndarray:
    """
    Verilen model ve tokenizer kullanarak bir metnin tek boyutlu cümle gömme vektörünü hesaplar.
    Token seviyesindeki gömmelerden ortalama alarak tek bir vektör oluşturur.
    
    Args:
        model: Hugging Face transformers kütüphanesinden bir AutoModel nesnesi
        tokenizer: Model ile uyumlu bir AutoTokenizer nesnesi
        text: Gömme vektörü oluşturulacak metin
        device_type: Hesaplamanın yapılacağı cihaz tipi ("cuda" veya "cpu")
    
    Returns:
        np.ndarray: Cümle gömme vektörünü içeren tek boyutlu numpy dizisi
    """
    # Girdi metnini tokenize et
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    
    # Girdileri model ile aynı cihaza taşı
    inputs = {k: v.to(device_type) for k, v in inputs.items()}
    
    # Model çıktılarını al - bellek tasarrufu için no_grad kullan
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Gömmeleri (son gizli durumları) al
    # Boyut: [batch_size, sequence_length, hidden_size]
    embeddings = outputs.last_hidden_state
    
    # Token gömmelerinin ortalamasını alarak cümle gömmesi oluştur
    # (1, sequence_length, hidden_size) -> (hidden_size,)
    sentence_embedding = embeddings.squeeze(0).mean(axis=0)
    
    # Numpy dizisine dönüştür ve döndür
    return sentence_embedding.cpu().numpy()
def get_row_embeddings(veri_satiri: pd.Series, model: AutoModel, tokenizer: AutoTokenizer, device_type: str="cuda"):
    """
    Verilen veri satırı için hem sentence hem de label gömmelerini hesaplar.
    
    Args:
        veri_satiri: Bir veri satırı
        model: Hugging Face transformers kütüphanesinden bir nesne
        tokenizer: Model ile uyumlu bir nesne
        device_type: Hesaplamanın yapılacağı cihaz tipi ("cuda" veya "cpu")
    
    Returns:
        dict: "sentence" ve "label" anahtarları altında hesaplanan gömme vektörlerini içeren sözlük
    """
    # Sentence gömmesini hesapla
    sentence_embedding = get_token_embedding(model, tokenizer, veri_satiri['sentence'], device_type)
    
    # Label gömmesini hesapla (label bir sayı ise string'e çevir)
    label_text = str(veri_satiri['label'])
    label_embedding = get_token_embedding(model, tokenizer, label_text, device_type)
    
    # Gömmeleri sözlük olarak döndür
    return {
        "sentence": sentence_embedding,
        "label": label_embedding
    }

def calculate_and_save_model_embeddings_specific_dataset(
    model_name: str,
    model: AutoModel, 
    tokenizer: AutoTokenizer, 
    veri_kumesi_adi: str,
    df: pd.DataFrame,
    model_dosya_adi_on_eki: str,
    veri_kumesi_adi_on_eki: str,
    device_type: str = "cuda"
):
    """
    Belirtilen model ve veri kümesi için gömmeleri hesaplar ve kaydeder.
    Varolan gömmeleri kontrol eder ve sadece hesaplanmayanları hesaplar.
    
    Args:
        model_name: Model adı (string)
        model: Hugging Face transformers kütüphanesinden bir AutoModel nesnesi
        tokenizer: Model ile uyumlu bir AutoTokenizer nesnesi
        veri_kumesi_adi: Veri kümesi adı (string)
        df: İşlenecek pandas DataFrame
        model_dosya_adi_on_eki: Model dosya adı ön eki
        veri_kumesi_adi_on_eki: Veri kümesi adı ön eki
        device_type: Hesaplamanın yapılacağı cihaz tipi ("cuda" veya "cpu")
    """
    # Modeli belirtilen cihaza taşı
    model.to(device_type)
    
    # Kaç gömmenin hesaplandığını kontrol et
    hesaplanan_sayisi = hesaplanan_gomme_sayisi_al(model_dosya_adi_on_eki, veri_kumesi_adi_on_eki)
    print(f"{model_name}-{veri_kumesi_adi} model-veri kümesi için {hesaplanan_sayisi} adet gömme önceden hesaplanmış.")
    
    # Tüm gömmeler zaten hesaplanmışsa, işlem yapma
    if hesaplanan_sayisi >= len(df):
        print("Tüm gömmeler zaten hesaplanmış.")
        return
    
    # Hesaplanacak gömmelerin aralığını belirle
    baslangic_indeksi = hesaplanan_sayisi
    toplam_ornek_sayisi = len(df)
    
    print(f"Toplam {toplam_ornek_sayisi - baslangic_indeksi} adet gömme hesaplanacak.")
    
    # Hesaplanmayan gömmeler için hesaplama yap
    for idx in range(baslangic_indeksi, toplam_ornek_sayisi):
        veri_satiri = df.iloc[idx]
        
        # Gömmeleri hesapla
        embedding_result = get_row_embeddings(veri_satiri, model, tokenizer, device_type)
        
        # Ek bilgileri ekle
        raw_embedding_dict = {
            "index": idx,
            "model_name": model_name,
            "dataset_name": veri_kumesi_adi,
            "sentence": veri_satiri['sentence'],
            "label": int(veri_satiri['label']),
            "sentence_embedding": embedding_result["sentence"].tolist(),
        }
        
        # Gömme sonucunu kaydet
        gomme_sonucu_ekle(raw_embedding_dict, model_dosya_adi_on_eki, veri_kumesi_adi_on_eki)
        
        # Her 10 örnekte bir ilerleme raporu ver
def get_token_embedding(model: AutoModel, tokenizer: AutoTokenizer, text: str, device_type: str = "cuda") -> np.ndarray:
    """
    Verilen model ve tokenizer kullanarak bir metnin tek boyutlu cümle gömme vektörünü hesaplar.
    Token seviyesindeki gömmelerden ortalama alarak tek bir vektör oluşturur.
    
    Args:
        model: Hugging Face transformers kütüphanesinden bir AutoModel nesnesi
        tokenizer: Model ile uyumlu bir AutoTokenizer nesnesi
        text: Gömme vektörü oluşturulacak metin
        device_type: Hesaplamanın yapılacağı cihaz tipi ("cuda" veya "cpu")
    
    Returns:
        np.ndarray: Cümle gömme vektörünü içeren tek boyutlu numpy dizisi
    """
    # Girdi metnini tokenize et
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    
    # Girdileri model ile aynı cihaza taşı
    inputs = {k: v.to(device_type) for k, v in inputs.items()}
    
    # Model çıktılarını al - bellek tasarrufu için no_grad kullan
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Gömmeleri (son gizli durumları) al
    # Boyut: [batch_size, sequence_length, hidden_size]
    embeddings = outputs.last_hidden_state
    
    # Token gömmelerinin ortalamasını alarak cümle gömmesi oluştur
    # (1, sequence_length, hidden_size) -> (hidden_size,)
    sentence_embedding = embeddings.squeeze(0).mean(axis=0)
    
    # Numpy dizisine dönüştür ve döndür
    return sentence_embedding.cpu().numpy()
def get_row_embeddings(veri_satiri: pd.Series, model: AutoModel, tokenizer: AutoTokenizer, device_type: str="cuda"):
    """
    Verilen veri satırı için hem sentence hem de label gömmelerini hesaplar.
    
    Args:
        veri_satiri: Bir veri satırı
        model: Hugging Face transformers kütüphanesinden bir nesne
        tokenizer: Model ile uyumlu bir nesne
        device_type: Hesaplamanın yapılacağı cihaz tipi ("cuda" veya "cpu")
    
    Returns:
        dict: "sentence" ve "label" anahtarları altında hesaplanan gömme vektörlerini içeren sözlük
    """
    # Sentence gömmesini hesapla
    sentence_embedding = get_token_embedding(model, tokenizer, veri_satiri['sentence'], device_type)
    
    # Label gömmesini hesapla (label bir sayı ise string'e çevir)
    label_text = str(veri_satiri['label'])
    label_embedding = get_token_embedding(model, tokenizer, label_text, device_type)
    
    # Gömmeleri sözlük olarak döndür
    return {
        "sentence": sentence_embedding,
        "label": label_embedding
    }

def calculate_and_save_model_embeddings_specific_dataset(
    model_name: str,
    model: AutoModel, 
    tokenizer: AutoTokenizer, 
    veri_kumesi_adi: str,
    df: pd.DataFrame,
    model_dosya_adi_on_eki: str,
    veri_kumesi_adi_on_eki: str,
    device_type: str = "cuda"
):
    """
    Belirtilen model ve veri kümesi için gömmeleri hesaplar ve kaydeder.
    Varolan gömmeleri kontrol eder ve sadece hesaplanmayanları hesaplar.
    
    Args:
        model_name: Model adı (string)
        model: Hugging Face transformers kütüphanesinden bir AutoModel nesnesi
        tokenizer: Model ile uyumlu bir AutoTokenizer nesnesi
        veri_kumesi_adi: Veri kümesi adı (string)
        df: İşlenecek pandas DataFrame
        model_dosya_adi_on_eki: Model dosya adı ön eki
        veri_kumesi_adi_on_eki: Veri kümesi adı ön eki
        device_type: Hesaplamanın yapılacağı cihaz tipi ("cuda" veya "cpu")
    """
    # Modeli belirtilen cihaza taşı
    model.to(device_type)
    
    # Kaç gömmenin hesaplandığını kontrol et
    hesaplanan_sayisi = hesaplanan_gomme_sayisi_al(model_dosya_adi_on_eki, veri_kumesi_adi_on_eki)
    print(f"{model_name}-{veri_kumesi_adi} model-veri kümesi için {hesaplanan_sayisi} adet gömme önceden hesaplanmış.")
    
    # Tüm gömmeler zaten hesaplanmışsa, işlem yapma
    if hesaplanan_sayisi >= len(df):
        print("Tüm gömmeler zaten hesaplanmış.")
        return
    
    # Hesaplanacak gömmelerin aralığını belirle
    baslangic_indeksi = hesaplanan_sayisi
    toplam_ornek_sayisi = len(df)
    
    print(f"Toplam {toplam_ornek_sayisi - baslangic_indeksi} adet gömme hesaplanacak.")
    
    # Hesaplanmayan gömmeler için hesaplama yap
    for idx in range(baslangic_indeksi, toplam_ornek_sayisi):
        veri_satiri = df.iloc[idx]
        
        # Gömmeleri hesapla
        embedding_result = get_row_embeddings(veri_satiri, model, tokenizer, device_type)
        
        # Ek bilgileri ekle
        raw_embedding_dict = {
            "index": idx,
            "model_name": model_name,
            "dataset_name": veri_kumesi_adi,
            "sentence": veri_satiri['sentence'],
            "label": int(veri_satiri['label']),
            "sentence_embedding": embedding_result["sentence"].tolist(),
        }
        
        # Gömme sonucunu kaydet
        gomme_sonucu_ekle(raw_embedding_dict, model_dosya_adi_on_eki, veri_kumesi_adi_on_eki)
        
        # Her 10 örnekte bir ilerleme raporu ver
        if (idx - baslangic_indeksi + 1) % 10 == 0 or idx == toplam_ornek_sayisi - 1:
            print(f"İlerleme: {idx - baslangic_indeksi + 1}/{toplam_ornek_sayisi - baslangic_indeksi} ({((idx - baslangic_indeksi + 1) / (toplam_ornek_sayisi - baslangic_indeksi)) * 100:.2f}%)", end="\r")
    
    print(f"Tüm gömmeler başarıyla hesaplandı ve kaydedildi.")

In [ ]:
def get_model_dosya_adi_on_eki(model_name: str) -> str:
    """
    Model dosya adı ön ekini döndürür.
    
    Args:
        model_name: Model adı (string)
    
    Returns:
        str: Model dosya adı ön eki
    """
    return model_name.lower().replace("/", "_")
def get_veri_kumesi_adi_on_eki(veri_kumesi_tipi: str, veri_kumesi_adi: str) -> str:
    """
    Veri kümesi adı ön ekini döndürür.
    
    Args:
        veri_kumesi_tipi: Veri kümesi tipi (egitim veya sinama)
        veri_kumesi_adi: Veri kümesi adı
    
    Returns:
        str: Veri kümesi adı ön eki
    """
    return veri_kumesi_tipi + "_" + veri_kumesi_adi.lower().replace(" ", "_")

## Gömmeleri hesapla

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Kullanılan cihaz tipi: {device}")

In [ ]:
model_names = [
    "intfloat/multilingual-e5-base",                                # 278M  - 33
    "ibm-granite/granite-embedding-107m-multilingual",              # 107M  - 48
    "intfloat/multilingual-e5-small",                               # 118M  - 36
    "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",  # 118M  - 62
    "shibing624/text2vec-base-multilingual",                        # 118M  - 77 
    ]
for i, model_name in enumerate(model_names):
    print(f"Model: {model_name} yükleniyor...")
    model = AutoModel.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    for veri_kumesi_tipi in veri_kumeleri.keys():
        for veri_kumesi_adi in veri_kumeleri[veri_kumesi_tipi].keys():
            islenecek_veri_kumesi = veri_kumeleri[veri_kumesi_tipi][veri_kumesi_adi]
            veri_kumesi_adi_on_eki = get_veri_kumesi_adi_on_eki(veri_kumesi_tipi, veri_kumesi_adi)
            calculate_and_save_model_embeddings_specific_dataset(
                model_name,
                model,
                tokenizer,
                veri_kumesi_adi,
                islenecek_veri_kumesi,
                get_model_dosya_adi_on_eki(model_name),
                veri_kumesi_adi_on_eki,
                device_type=device
            )
    print(f"Model: {model_name} yüklendi ve gömme hesaplamaları tamamlandı. ({i + 1}/{len(model_names)})")

## Gömmeleri yükle

Aşağıdaki yapıda yüklenir. Eğitim ve sınama kümeleri altında farklı veri kümeleri vardır. onun da altında her veri kümesi için farklı gömme modelleriyle elde edilen gömme sonuçları bulunmaktadır.
```json
{
    "veri_kumesi_tipi": "sinama_kumeleri",
    "gommeler": [
        {
            "veri_kumesi_adi": "sst2",
            "intfloat/multilingual-e5-base": [],
            "ibm-granite/granite-embedding-107m-multilingual": [],
            "intfloat/multilingual-e5-small": [],
            "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2": [],
            "shibing624/text2vec-base-multilingual": []
        },
        {
            "veri_kumesi_adi": "cola",
            "intfloat/multilingual-e5-base": [],
            "ibm-granite/granite-embedding-107m-multilingual": [],
            "intfloat/multilingual-e5-small": [],
            "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2": [],
            "shibing624/text2vec-base-multilingual": []
        }
    ]
}
{
    "veri_kumesi_tipi": "egiti_kumeleri",
    "gommeler": [
        {
            "veri_kumesi_adi": "sst2",
            "intfloat/multilingual-e5-base": [],
            "ibm-granite/granite-embedding-107m-multilingual": [],
            "intfloat/multilingual-e5-small": [],
            "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2": [],
            "shibing624/text2vec-base-multilingual": []
        },
        {
            "veri_kumesi_adi": "cola",
            "intfloat/multilingual-e5-base": [],
            "ibm-granite/granite-embedding-107m-multilingual": [],
            "intfloat/multilingual-e5-small": [],
            "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2": [],
            "shibing624/text2vec-base-multilingual": []
        }
    ]
}
```

In [ ]:
veri_kumeleri_gommeleri = []
# sırayla önce sınama sonra eğitim kümesi için gömme yükle
for veri_kumesi_tipi in veri_kumeleri.keys():
    tip_gommeleri = {}
    tip_gommeleri["veri_kumesi_tipi"] = veri_kumesi_tipi
    tip_gommeleri["gommeler"] = []
    # eğitim/sınama
    tipe_gore_veri_kumeleri = veri_kumeleri[veri_kumesi_tipi]
    # sst2 ve cola'la gez
    for veri_kumesi_adi in tipe_gore_veri_kumeleri.keys():
        # sst2/cola'nın tüm modellerdeki gömmeleri
        veri_kumesi_gommeleri = {}
        veri_kumesi_gommeleri["veri_kumesi_adi"] = veri_kumesi_adi
        for i, model_name in enumerate(model_names):
            veri_kumesi_adi_on_eki = get_veri_kumesi_adi_on_eki(veri_kumesi_tipi, veri_kumesi_adi)
            model_dosya_adi_on_eki = get_model_dosya_adi_on_eki(model_name)
            # model-veri kümesi için gömme yükle (tipe göre eğitim/sınama)
            veri_kumesi_gommeleri[model_name] = gomme_bilgisi_oku(model_dosya_adi_on_eki, veri_kumesi_adi_on_eki)
            print(f"{model_name}-{veri_kumesi_adi} model-veri kümesi için {len(veri_kumesi_gommeleri[model_name])} adet gömme yüklendi. ({i + 1}/{len(model_names)})")
        tip_gommeleri["gommeler"].append(veri_kumesi_gommeleri)
    veri_kumeleri_gommeleri.append(tip_gommeleri)
print("Tüm gömme verileri yüklendi.")

## Yüklenen gömmeleri modeli veri kümesi adı ve tipine göre getirme fonksiyonu

In [ ]:
def gomme_getir(veri_kumeleri_gommeleri: list, veri_kumesi_adi: str, model_name: str, veri_kumesi_tipi: str) -> list:
    """
    Verilen veri kümesi adı, model adı ve veri kümesi tipine göre gömme vektörlerini döndürür.

    Args:
        veri_kumeleri_gommeleri: Gömme verileri
        veri_kumesi_adi: Veri kümesi adı
        model_name: Model adı
        veri_kumesi_tipi: Veri kümesi tipi
    Returns:
        list: Gömme vektörlerini içeren liste
    """
    if veri_kumeleri_gommeleri[0]["veri_kumesi_tipi"] == veri_kumesi_tipi:
        tip_gommeleri = veri_kumeleri_gommeleri[0]["gommeler"]
    else:
        tip_gommeleri = veri_kumeleri_gommeleri[1]["gommeler"]
    for veri_kumesi_gommeleri in tip_gommeleri:
        if veri_kumesi_gommeleri["veri_kumesi_adi"] == veri_kumesi_adi:
            return veri_kumesi_gommeleri[model_name]
    return None

## Model tipine göre eğitme, oluşturma ve sınama fonksiyonları

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [ ]:
def model_getir(model_sinifi, seed=42):
    """
    Gömme boyutuna ve model tipine göre model oluşturup döndürür.
    
    Parametreler:
        model_sinifi: Kullanılacak model sınıfı (RandomForestClassifier, LogisticRegression, SVC vb.)
        seed: Rastgelelik için kullanılacak tohum değeri
        
    Dönüş:
        Oluşturulan model nesnesi
    """
    if model_sinifi == RandomForestClassifier:
        return RandomForestClassifier(n_estimators=100, random_state=seed)
    elif model_sinifi == LogisticRegression:
        return LogisticRegression(max_iter=1000, random_state=seed)
    elif model_sinifi == SVC:
        return SVC(probability=True, random_state=seed)
    else:
        raise ValueError("Desteklenmeyen model sınıfı")

def model_egit(model, egitim_gommeleri):
    """
    Verilen modeli eğitim gömmelerini kullanarak eğitir.
    
    Parametreler:
        model: Eğitilecek model nesnesi
        egitim_gommeleri: Eğitim için kullanılacak gömme vektörleri listesi
        
    Dönüş:
        Eğitilmiş model nesnesi
    """
    # Eğitim verilerinden X (özellikler) ve y (etiketler) oluşturma
    X_train = np.vstack([gomme["sentence_embedding"] for gomme in egitim_gommeleri])
    y_train = np.array([gomme["label"] for gomme in egitim_gommeleri])
    
    # Modeli eğit
    model.fit(X_train, y_train)
    
    return model

def model_basari_hesapla(model, sinama_gommeleri):
    """
    Eğitilmiş modelin sınama gömmelerindeki başarı metriklerini hesaplar.
    
    Parametreler:
        model: Değerlendirilecek eğitilmiş model nesnesi
        sinama_gommeleri: Test için kullanılacak gömme vektörleri listesi
        
    Dönüş:
        dict: Accuracy, precision, recall, f1 ve diğer metrikleri içeren sözlük
    """
    # Test verilerinden X (özellikler) ve y (etiketler) oluşturma
    X_test = np.vstack([gomme["sentence_embedding"] for gomme in sinama_gommeleri])
    y_test = np.array([gomme["label"] for gomme in sinama_gommeleri])
    
    # Tahminler
    y_pred = model.predict(X_test)
    
    # Metrikler
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)
    
    # Confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    
    # Sonuçları sözlük olarak döndür
    sonuc = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'confusion_matrix': cm.tolist()
    }
    
    return sonuc

## Modelleri eğit ve sına

In [ ]:
ml_model_tipleri = [RandomForestClassifier, LogisticRegression, SVC]
ml_model_tipleri_str = ["Rastgele Orman (Random Forest)", "Lojistik Regresyon (Logistic Regression)", "Destek Vektör Makineleri (Support Vector Machines)"]
for veri_kumesi_adi in veri_kumesi_adlari:
    for model_name in model_names:
        for i in range(len(ml_model_tipleri)):
            if not gomme_basarimi_var_mi(ml_model_tipleri_str[i], model_name, veri_kumesi_adi):
                egitim_gommeleri = gomme_getir(veri_kumeleri_gommeleri, veri_kumesi_adi, model_name, "egitim_kumeleri")
                sinama_gommeleri = gomme_getir(veri_kumeleri_gommeleri, veri_kumesi_adi, model_name, "sinama_kumeleri")
                print(f"{veri_kumesi_adi} veri kümesi için {model_name} modeli ve {ml_model_tipleri_str[i]} için doğruluk değeri hesaplanıyor...", end="\r")
                model = model_getir(ml_model_tipleri[i])
                model = model_egit(model, egitim_gommeleri)
                basari_dict = model_basari_hesapla(model, sinama_gommeleri)
                gomme_basarimi_kaydet(basari_dict, ml_model_tipleri_str[i], model_name, veri_kumesi_adi)
                print(f"{veri_kumesi_adi} veri kümesi için {model_name} modeli ve {ml_model_tipleri_str[i]} için doğruluk değeri hesaplandı. ({i + 1}/{len(ml_model_tipleri)})", end="\r")
            else:
                print(f"{veri_kumesi_adi} veri kümesi için {model_name} modeli ve {ml_model_tipleri_str[i]} için doğruluk değeri zaten hesaplanmış.")
        print(f"\n{veri_kumesi_adi} veri kümesi için {model_name} modeli için doğruluk değerleri hesaplandı. ({model_names.index(model_name) + 1}/{len(model_names)})")
    print(f"{veri_kumesi_adi} veri kümesi için tüm model doğruluk değerleri hesaplandı.")

# Çizdirme işlemleri

In [ ]:
from cizdir import gomme_modeli_basari_puanlari_cizdir, gomme_modeli_f1_puanlari_cizdir, \
    gomme_modeli_recall_puanlari_cizdir, gomme_modeli_basari_puanlari_cizdir, gomme_modeli_conf_matris_cizdir
from dizin_yardimcisi import gomme_basarimlari_oku

In [ ]:

renkler = ['#8B4513', '#8B0000', '#006400'] 
gomme_sonuclari = gomme_basarimlari_oku()
for veri_kumesi_adi in veri_kumesi_adlari:
    gomme_modeli_basari_puanlari_cizdir(f"{veri_kumesi_adi} Sonuçları", gomme_sonuclari[veri_kumesi_adi], veri_kumesi_adi)
    gomme_modeli_f1_puanlari_cizdir(f"{veri_kumesi_adi} Sonuçları", gomme_sonuclari[veri_kumesi_adi], veri_kumesi_adi)
    gomme_modeli_recall_puanlari_cizdir(f"{veri_kumesi_adi} Sonuçları", gomme_sonuclari[veri_kumesi_adi], veri_kumesi_adi)
    gomme_modeli_basari_puanlari_cizdir(f"{veri_kumesi_adi} Sonuçları", gomme_sonuclari[veri_kumesi_adi], veri_kumesi_adi)
    for ml_model_name in ml_model_tipleri_str:
        for model_name in model_names:
            gomme_modeli_conf_matris_cizdir(f"{veri_kumesi_adi} Sonuçları", gomme_sonuclari[veri_kumesi_adi], veri_kumesi_adi,
                                             ml_model_name, model_name, get_model_dosya_adi_on_eki(model_name))
    print(f"{model_name}-{veri_kumesi_adi} model-veri kümesi için başarı puanları çizdirildi.")